In [183]:
import numpy as np
import tensorflow as tf
import pandas as pd
from datetime import datetime

In [184]:
#sigmoid activation function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def sigmoid_deriv(x):
    return x * (1 - x)

In [185]:
#relu activation function
def relu(x):
    return np.maximum(0,x)
def relu_deriv(x):
    return np.greater(x, 0)

In [186]:
def diff(target_label, predicted_label):
    return (target_label[0][0] - predicted_label[0][0])

In [197]:
class NeuralNetwork:
    def __init__(self, input_nodes, hidden_nodes, output_nodes):
        self.w_ih = np.random.randn(hidden_nodes, input_nodes)
        self.w_ho = np.random.randn(output_nodes, hidden_nodes)
        self.bh = np.random.randn(hidden_nodes, 1)
        self.bo = np.random.randn(output_nodes, 1)
        self.lr = 0.01
    def feed_forward(self, X, Y):
        hidden_layer = sigmoid(np.dot(self.w_ih, X) + self.bh)
        output_layer = sigmoid(np.dot(self.w_ho, hidden_layer) + self.bo)
        prediction = 0 if output_layer[0][0] < 0.5 else 1
        return hidden_layer, output_layer, diff(Y, np.array([[prediction]])), prediction
    def backpropogate(self, X, o_errors, o_layer, h_layer):
        #calculating output gradient
        o_gradients = sigmoid_deriv(o_layer)
        o_gradients = np.multiply(o_gradients, o_errors)
        o_gradients = np.multiply(o_gradients, self.lr)
        #calculate output weight deltas
        h_layer_T = h_layer.transpose()
        w_ho_delta = np.dot(o_gradients, h_layer_T)
        self.w_ho += w_ho_delta
        self.bo += o_gradients
        who_t = self.w_ho.transpose()
        #calculating hidden error
        h_errors = np.dot(who_t, o_errors)
        #calculating hidden gradient
        h_gradients = sigmoid_deriv(h_layer)
        h_gradients = np.multiply(h_gradients, h_errors)
        h_gradients = np.multiply(h_gradients, self.lr)
        
        X_T = np.transpose(X)
        w_ih_delta = np.dot(h_gradients, X_T)
        self.w_ih += w_ih_delta
        self.bh += h_gradients
    
    def train(self, X, Y, epochs=10, num_samples=1000):
        for epoch in range(0, epochs):
            error_s = 0
            correct = 0
            for i in range(num_samples):
                index = np.random.choice(X.shape[0], 1)
                x_train = X[index].reshape(6, 1)
                y_train = Y[index].reshape(1, 1)
                h_layer, o_layer, error, prediction = self.feed_forward(x_train, y_train)
                if prediction == y_train[0][0]:
                    correct += 1
                error_s += error
                error = np.array([[error]])
                self.backpropogate(x_train, error, o_layer, h_layer)
            avg_error = error_s / num_samples
            avg_acc = correct / num_samples
            print(f"Error after epoch {epoch + 1} is: {round(avg_error, 3)} and accuracy is: {avg_acc}")
    def evaluate(self, X, Y):
        num_samples = X.shape[0]
        correct = 0
        for i in range(num_samples):
            x_val, y_val = X[i].reshape(6, 1), Y[i].reshape(1, 1)
            _, __, ___, prediction = self.feed_forward(x_val, y_val)
            if prediction == y_val[0][0]:
                correct += 1
        val_acc = correct / num_samples
        return val_acc

In [188]:
def getXY(filename):
    training_data_filename = "occupancy_data/datatraining.csv"
    data = pd.read_csv(training_data_filename)
    data = data.drop(labels=['id'], axis=1)
    data['date'] = [datetime.strptime(d, "%Y-%m-%d %H:%M:%S").timestamp() for d in data['date']] #change to date to seconds
    X = data.drop(labels=["Occupancy"], axis=1)
    Y = data['Occupancy']
    #normalize the dataset 
    X['date'] = tf.keras.utils.normalize(np.array(X['date']))[0]
    X['Temperature'] = tf.keras.utils.normalize(np.array(X['Temperature']))[0]
    X['Humidity'] = tf.keras.utils.normalize(np.array(X['Humidity']))[0]
    X['Light'] = tf.keras.utils.normalize(np.array(X['Light']))[0]
    X['CO2'] = tf.keras.utils.normalize(np.array(X['CO2']))[0]
    X['HumidityRatio'] = tf.keras.utils.normalize(np.array(X['HumidityRatio']))[0]
    X = np.array(X)
    Y = np.array(Y).reshape(-1, 1)
    return X, Y

In [189]:
training_data_filename = "occupancy_data/datatraining.csv"
X, Y = getXY(training_data_filename)

In [190]:
X.shape, Y.shape

((8143, 6), (8143, 1))

In [191]:
X, Y

(array([[0.01107983, 0.01244298, 0.01148291, 0.02066048, 0.0116999 ,
         0.01342832],
        [0.01107983, 0.01242688, 0.01148101, 0.02083022, 0.01158229,
         0.01340157],
        [0.01107983, 0.01242688, 0.01147154, 0.02066048, 0.01157418,
         0.01339043],
        ...,
        [0.01108364, 0.01132644, 0.01519784, 0.02099997, 0.01295303,
         0.01567707],
        [0.01108364, 0.01132644, 0.01526732, 0.02099997, 0.0133072 ,
         0.01574939],
        [0.01108364, 0.01132644, 0.01524205, 0.02167895, 0.01331802,
         0.01572309]]),
 array([[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]], dtype=int64))

In [207]:
nn = NeuralNetwork(6, 128, 1)

In [208]:
#training the model
nn.train(X, Y, 10, X.shape[0])

Error after epoch 1 is: 0.004 and accuracy is: 0.8085472184698514
Error after epoch 2 is: 0.0 and accuracy is: 0.8381431904703426
Error after epoch 3 is: -0.0 and accuracy is: 0.8800196487780916
Error after epoch 4 is: 0.0 and accuracy is: 0.8728969667198822
Error after epoch 5 is: 0.0 and accuracy is: 0.8719145278153014
Error after epoch 6 is: -0.0 and accuracy is: 0.9301240329117033
Error after epoch 7 is: 0.0 and accuracy is: 0.9506324450448238
Error after epoch 8 is: 0.0 and accuracy is: 0.9521061034016947
Error after epoch 9 is: 0.0 and accuracy is: 0.9560358590200172
Error after epoch 10 is: 0.0 and accuracy is: 0.9599656146383396


In [201]:
test1_filename = "occupancy_data/datatest.csv"
test2_filename = "occupancy_data/datatest2.csv"

In [202]:
X_test1, Y_test1 = getXY(test1_filename)

In [211]:
#testing the model performance with datatest.csv
print("Accuracy:", nn.evaluate(X_test1, Y_test1))

Accuracy: 0.9831757337590569


In [204]:
X_test2, Y_test2 = getXY(test2_filename)

In [212]:
#testing the model performance with datatest2.csv
nn.evaluate(X_test2, Y_test2)

0.9831757337590569